In [1]:
# Wrangling Data
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
#pip install geopy
from geopy.geocoders import Nominatim #Convert adress to Long lat

import json
import requests
import urllib.request, json
from pandas.io.json import json_normalize

#Matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

#K-means
from sklearn.cluster import KMeans


#!conda install -c conda-forge folium=0.5.0 --yes
#pip install folium=0.5.0
import folium

print('Library imported.')

Library imported.


In [62]:
url_a = ("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# read url_a
Demo_data = pd.read_html(url_a, header=0)
#Demo_data[0]

#delete NaN row
myData = Demo_data[0].dropna()
#myData = myData.drop(columns = ['Postal code']) #drop Postal Code columns
myData.sort_values(by = ['Borough'], ascending = True, inplace = True) #Sort my data by Borough
myData = myData.reset_index(drop = True)
neigh_columns = myData.Neighborhood.str.split('/',expand = True,)
myData = pd.concat([myData['Borough'], neigh_columns], axis=1, sort=False)
myData.set_index(['Borough'], inplace = True)
myData
#================================
#list_columns = ['Borough', 'Borough_Latitude', 'Borough_Longitude']
#df = pd.DataFrame(columns=list_columns)
#ind = myData['Borough'].unique()
#df['Borough'] = ind
#df.fillna(0, inplace=True)
#df
#================================
ind = myData.index.unique()
df = myData.loc[ind]
dfStack = df.stack()
dfStack = dfStack.to_frame()
dfStack.rename(columns = {0:"Neighborhood"}, inplace = True)
dfStack.reset_index(inplace = True)
dfStack = dfStack.drop(columns = ['level_1']) #drop level_1 columns
dfStack
#=================================
#len(myData.index.unique())



<ipython-input-62-64d7e7a02dd6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myData.sort_values(by = ['Borough'], ascending = True, inplace = True) #Sort my data by Borough


,Borough,Neighborhood
0,Central Toronto,Summerhill West
1,Central Toronto,Rathnelly
2,Central Toronto,South Hill
3,Central Toronto,Forest Hill SE
4,Central Toronto,Deer Park
5,Central Toronto,Davisville
6,Central Toronto,Moore Park
7,Central Toronto,Summerhill East
8,Central Toronto,Forest Hill North & West
9,Central Toronto,The Annex


In [63]:
dfStack_CT = dfStack.iloc[:15]
dfStack_DT = dfStack.iloc[16:53]
dfStack_ET = dfStack.iloc[54:60]
dfStack_EY = dfStack.iloc[61:66]
dfStack_EC = dfStack.iloc[67:111]
dfStack_NY = dfStack.iloc[112:148]
dfStack_SC = dfStack.iloc[149:186]
dfStack_WT = dfStack.iloc[187:199]
dfStack_YK = dfStack.iloc[200:207]

In [64]:
dfStack.shape

(208, 2)

In [65]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfStack['Borough'].unique()),
        dfStack['Neighborhood'].shape[0]
    )
)

The dataframe has 10 boroughs and 208 neighborhoods.


In [73]:
lat_col = []
long_col =[]
for borough_name, neigh_name in zip (dfStack_CT['Borough'], dfStack_CT['Neighborhood']):
    address = borough_name+','+neigh_name
    geolocator = Nominatim(user_agent = "on_explorer")
    try:
        location = geolocator.geocode(address)
        lat_col.append(location.latitude)
        long_col.append(location.longitude)
        
    except(AttributeError,NameError,ValueError):
        lat_col.append(np.NaN)
        long_col.append(np.NaN) 
        
dfStack_CT['Latitude'] = lat_col
dfStack_CT['Longitude'] = long_col
dfStack_CT

<ipython-input-73-d834e46f2b1c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStack_CT['Latitude'] = lat_col
<ipython-input-73-d834e46f2b1c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStack_CT['Longitude'] = long_col


,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,Summerhill West,43.681678,-79.390504
1,Central Toronto,Rathnelly,43.678395,-79.400822
2,Central Toronto,South Hill,43.677926,-79.405767
3,Central Toronto,Forest Hill SE,NaN,NaN
4,Central Toronto,Deer Park,43.688090,-79.394093
5,Central Toronto,Davisville,43.697936,-79.397291
6,Central Toronto,Moore Park,43.690388,-79.383297
7,Central Toronto,Summerhill East,43.681678,-79.390504
8,Central Toronto,Forest Hill North & West,43.693559,-79.413902
9,Central Toronto,The Annex,43.670338,-79.407117


In [76]:
dfStack_CT.dropna(inplace = True)

<ipython-input-76-ce76ed38f411>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStack_CT.dropna(inplace = True)


In [77]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [78]:
#### Create a map of Toronto with neighborhoods superimposed on top.

In [80]:
# create map of Central Toronto using latitude and longitude values
map_centralToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfStack_CT['Latitude'], dfStack_CT['Longitude'], dfStack_CT['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_centralToronto)  
    
map_centralToronto

In [81]:
#### Define Foursquare Credentials and Version

In [83]:
#Get number of borough and neiborhoods
#print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(dfStack['Borough'].unique()), dfStack.shape))
LIMIT = 100 # limit of number 
radius = 500 # define radius
Cl_ID ='PKDDMXMWQQEBO3QSFTXG1WHFUTHY1KSR3FPRMAGDQETCD0UN'
Cl_SECRET = 'PL5AMKK2C3GDJWX5Q2HIIOLSLHBL0KER5LTKR4ZSW3V11WOW'
VERSION = '20180605'


print('My credentails:')
print('CLIENT_ID: ' + Cl_ID)
print('CLIENT_SECRET:' + Cl_SECRET)

My credentails:
CLIENT_ID: PKDDMXMWQQEBO3QSFTXG1WHFUTHY1KSR3FPRMAGDQETCD0UN
CLIENT_SECRET:PL5AMKK2C3GDJWX5Q2HIIOLSLHBL0KER5LTKR4ZSW3V11WOW


In [84]:
#### Let's explore the first neighborhood in our dataframe.

In [85]:
dfStack_CT.loc[0, 'Neighborhood']

'Summerhill West '

In [86]:
neighborhood_latitude = dfStack_CT.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfStack_CT.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfStack_CT.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Summerhill West  are 43.6816776, -79.3905037.


In [87]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    Cl_ID, 
    Cl_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PKDDMXMWQQEBO3QSFTXG1WHFUTHY1KSR3FPRMAGDQETCD0UN&client_secret=PL5AMKK2C3GDJWX5Q2HIIOLSLHBL0KER5LTKR4ZSW3V11WOW&v=20180605&ll=43.6816776,-79.3905037&radius=500&limit=100'

In [88]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e98d514aba2971ba54f7bc2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Summerhill',
  'headerFullLocation': 'Summerhill, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 43.6861776045, 'lng': -79.38429286380546},
   'sw': {'lat': 43.6771775955, 'lng': -79.39671453619454}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aea405ef964a52082ba21e3',
       'name': 'LCBO',
       'location': {'address': '10 Scrivener Sq.',
        'crossStreet': 'at Yonge St.',
        'lat': 43.68149682937831,
        'lng': -79.39126079276781,
        'labeledLatLngs': [{'label': 'display',
   

In [89]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [90]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,LCBO,Liquor Store,43.681497,-79.391261
1,Boxcar Social,Coffee Shop,43.681750,-79.391447
2,Waxon Waxbar,Spa,43.682824,-79.391971
3,Terroni,Italian Restaurant,43.679870,-79.390525
4,Manipedi Spa,Spa,43.682626,-79.391879


In [91]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


In [92]:
## 2. Explore Neighborhoods in Central Toronto

In [ ]:
#### Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius
        # create URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        Cl_ID, 
        Cl_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT)
        url # display URL
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [97]:
Central_Downtown_venues = getNearbyVenues(names=dfStack_CT['Neighborhood'],
                                   latitudes=dfStack_CT['Latitude'],
                                   longitudes=dfStack_CT['Longitude']
                                  )

Summerhill West 
 Rathnelly 
 South Hill 
 Deer Park
Davisville
Moore Park 
 Summerhill East
Forest Hill North & West
The Annex 
 North Midtown 
 Yorkville
Davisville North
Roselawn
North Toronto West


In [99]:
print(Central_Downtown_venues.shape)
Central_Downtown_venues.head()

(588, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Summerhill West,43.681678,-79.390504,LCBO,43.681497,-79.391261,Liquor Store
1,Summerhill West,43.681678,-79.390504,Boxcar Social,43.681750,-79.391447,Coffee Shop
2,Summerhill West,43.681678,-79.390504,Waxon Waxbar,43.682824,-79.391971,Spa
3,Summerhill West,43.681678,-79.390504,Terroni,43.679870,-79.390525,Italian Restaurant
4,Summerhill West,43.681678,-79.390504,Manipedi Spa,43.682626,-79.391879,Spa


In [100]:
Central_Downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Deer Park,42,42,42,42,42,42
North Midtown,42,42,42,42,42,42
Rathnelly,42,42,42,42,42,42
South Hill,42,42,42,42,42,42
Summerhill East,42,42,42,42,42,42
Yorkville,42,42,42,42,42,42
Davisville,42,42,42,42,42,42
Davisville North,42,42,42,42,42,42
Forest Hill North & West,42,42,42,42,42,42


In [101]:
print('There are {} uniques categories.'.format(len(Central_Downtown_venues['Venue Category'].unique())))

There are 34 uniques categories.


In [ ]:
## 3. Analyze Each Neighborhood

In [102]:
# one hot encoding
CT_onehot = pd.get_dummies(Central_Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CT_onehot['Neighborhood'] = Central_Downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [CT_onehot.columns[-1]] + list(CT_onehot.columns[:-1])
CT_onehot = CT_onehot[fixed_columns]

CT_onehot.head()

,Wine Bar,American Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Store,Building,Butcher,Café,Coffee Shop,Cosmetics Shop,Dessert Shop,French Restaurant,Furniture / Home Store,Gift Shop,Gourmet Shop,Grocery Store,Gym / Fitness Center,Italian Restaurant,Juice Bar,Liquor Store,Neighborhood,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Pub,Spa,Sporting Goods Shop,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Summerhill West,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Summerhill West,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Summerhill West,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Summerhill West,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Summerhill West,0,0,0,0,0,1,0,0,0,0


In [103]:
CT_onehot.shape

(588, 34)

In [104]:
CT_grouped = CT_onehot.groupby('Neighborhood').mean().reset_index()
CT_grouped

,Neighborhood,Wine Bar,American Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Store,Building,Butcher,Café,Coffee Shop,Cosmetics Shop,Dessert Shop,French Restaurant,Furniture / Home Store,Gift Shop,Gourmet Shop,Grocery Store,Gym / Fitness Center,Italian Restaurant,Juice Bar,Liquor Store,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Pub,Spa,Sporting Goods Shop,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Deer Park,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
1,North Midtown,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
2,Rathnelly,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
3,South Hill,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
4,Summerhill East,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
5,Yorkville,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
6,Davisville,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
7,Davisville North,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
8,Forest Hill North & West,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381
9,Moore Park,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.071429,0.02381,0.02381,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.071429,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.047619,0.02381,0.02381


In [105]:
num_top_venues = 5

for hood in CT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = CT_grouped[CT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Deer Park----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3    Sushi Restaurant  0.05
4                 Spa  0.05


---- North Midtown ----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3    Sushi Restaurant  0.05
4                 Spa  0.05


---- Rathnelly ----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3    Sushi Restaurant  0.05
4                 Spa  0.05


---- South Hill ----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3    Sushi Restaurant  0.05
4                 Spa  0.05


---- Summerhill East----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3    Sushi Restaurant  0.05
4                 Spa  0.05


---- Yorkville----
                venue 

In [106]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [107]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CT_grouped['Neighborhood']

for ind in np.arange(CT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Deer Park,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
1,North Midtown,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
2,Rathnelly,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
3,South Hill,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
4,Summerhill East,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher


In [ ]:
## 4. Cluster Neighborhoods

In [108]:
# set number of clusters
kclusters = 5

CT_grouped_clustering = CT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

<ipython-input-108-2a4e28d39625>:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CT_grouped_clustering)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [109]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CT_merged = dfStack_CT

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
CT_merged = CT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

CT_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Summerhill West,43.681678,-79.390504,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
1,Central Toronto,Rathnelly,43.678395,-79.400822,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
2,Central Toronto,South Hill,43.677926,-79.405767,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
4,Central Toronto,Deer Park,43.688090,-79.394093,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
5,Central Toronto,Davisville,43.697936,-79.397291,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher


In [110]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CT_merged['Latitude'], CT_merged['Longitude'], CT_merged['Neighborhood'], CT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
## 5. Examine Clusters

In [111]:
CT_merged.loc[CT_merged['Cluster Labels'] == 0, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Summerhill West,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
1,Rathnelly,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
2,South Hill,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
4,Deer Park,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
5,Davisville,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
6,Moore Park,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
7,Summerhill East,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
8,Forest Hill North & West,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
9,The Annex,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
10,North Midtown,Coffee Shop,Italian Restaurant,Sushi Restaurant,Spa,Café,French Restaurant,Vegetarian / Vegan Restaurant,Building,Cosmetics Shop,Butcher
